In [49]:
from collections import defaultdict
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
import myfm
from myfm import RelationBlock
from scipy import sparse as sps

from sklearn.model_selection import train_test_split
import pandas as pd

def load_cil(dataset="split"):

    file = "data_train" if dataset != "test" else "sampleSubmission"
    data = pd.read_csv(f'../data/{file}.csv', index_col=0)
    data['user'] = data.index.str.split('_').str[0].str[1:].astype('int32')
    data['movie'] = data.index.str.split('_').str[1].str[1:].astype('int32')
    data.rename(columns={'Prediction': 'rating'}, inplace=True)
    data['rating'] = data['rating'].astype('uint8')
    data = data[['user', 'movie', 'rating']]

    data['user'] = data['user']
    data['movie'] = data['movie']
    # print("Subtracted {} from user and movie".format(1))

    user_num = 10000  # int(data['user'].max() + 1)
    movie_num = 1000  # int(data['movie'].max() + 1)
    print("User num: {}, Movie num: {}".format(user_num, movie_num))

    train_data = val_data = None
    if dataset == "test":
        val_data = data
    elif dataset == "train":
        train_data = data
    else:
        train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

    return train_data, val_data, user_num, movie_num

In [50]:
train_data, val_data, user_num, movie_num = load_cil("train")

User num: 10000, Movie num: 1000


In [55]:
_, val_data, user_num, movie_num = load_cil("test")

User num: 10000, Movie num: 1000


In [51]:
# index "0" is reserved for unknown ids.
user_to_index = defaultdict(lambda : 0, { uid: i+1 for i,uid in enumerate(np.unique(train_data.user)) })
movie_to_index = defaultdict(lambda: 0, { mid: i+1 for i,mid in enumerate(np.unique(train_data.movie))})
USER_ID_SIZE = len(user_to_index) + 1
MOVIE_ID_SIZE = len(movie_to_index) + 1

In [52]:
# The flags to control the included features.
use_iu = True # use implicit user feature
use_ii = True # use implicit item feature

movie_vs_watched = dict()
user_vs_watched = dict()
for row in train_data.itertuples():
    user_id = row.user
    movie_id = row.movie
    movie_vs_watched.setdefault(movie_id, list()).append(user_id)
    user_vs_watched.setdefault(user_id, list()).append(movie_id)

In [53]:
# given user/movie ids, add additional infos and return it as sparse
def augment_user_id(user_ids):
    Xs = []
    X_uid = sps.lil_matrix((len(user_ids), USER_ID_SIZE))
    for index, user_id in enumerate(user_ids):
        X_uid[index, user_to_index[user_id]] = 1
    Xs.append(X_uid)
    if use_iu:
        X_iu = sps.lil_matrix((len(user_ids), MOVIE_ID_SIZE))
        for index, user_id in enumerate(user_ids):
            watched_movies = user_vs_watched.get(user_id, [])
            normalizer = 1 / max(len(watched_movies), 1) ** 0.5
            for uid in watched_movies:
                X_iu[index, movie_to_index[uid]] = normalizer
        Xs.append(X_iu)
    return sps.hstack(Xs, format='csr')

def augment_movie_id(movie_ids):
    Xs = []
    X_movie = sps.lil_matrix((len(movie_ids), MOVIE_ID_SIZE))
    for index, movie_id in enumerate(movie_ids):
        X_movie[index, movie_to_index[movie_id]] = 1
    Xs.append(X_movie)

    if use_ii:
        X_ii = sps.lil_matrix((len(movie_ids), USER_ID_SIZE))
        for index, movie_id in enumerate(movie_ids):
            watched_users = movie_vs_watched.get(movie_id, [])
            normalizer = 1 / max(len(watched_users), 1) ** 0.5
            for uid in watched_users:
                X_ii[index, user_to_index[uid]] = normalizer
        Xs.append(X_ii)


    return sps.hstack(Xs, format='csr')

In [56]:
train_uid_unique, train_uid_index = np.unique(train_data.user, return_inverse=True)
train_mid_unique, train_mid_index = np.unique(train_data.movie, return_inverse=True)
user_data_train = augment_user_id(train_uid_unique)
movie_data_train = augment_movie_id(train_mid_unique)

test_uid_unique, test_uid_index = np.unique(val_data.user, return_inverse=True)
test_mid_unique, test_mid_index = np.unique(val_data.movie, return_inverse=True)
user_data_test = augment_user_id(test_uid_unique)
movie_data_test = augment_movie_id(test_mid_unique)

In [57]:
block_user_train = RelationBlock(train_uid_index, user_data_train)
block_movie_train = RelationBlock(train_mid_index, movie_data_train)
block_user_test = RelationBlock(test_uid_index, user_data_test)
block_movie_test = RelationBlock(test_mid_index, movie_data_test)

In [58]:
FM_RANK = 10

FEATURE_COLUMNS = ['user', 'movie']
ohe = OneHotEncoder(handle_unknown='ignore')

X_train = ohe.fit_transform(train_data[FEATURE_COLUMNS])
X_test = ohe.transform(val_data[FEATURE_COLUMNS])
y_train = train_data.rating.values
y_test = val_data.rating.values

In [59]:
fm_rb = myfm.MyFMRegressor(rank=FM_RANK).fit(
    X_train, y_train,
    X_rel=[block_user_train],
    n_iter=512, n_kept_samples=200
)

alpha = 1.13 w0 = 3.47 : 100%|██████████| 512/512 [05:27<00:00,  1.56it/s]


In [48]:
test_prediction = fm_rb.predict(
    X_test,
    X_rel=[block_user_test]
)

rmse = ((y_test - test_prediction) ** 2).mean() ** 0.5
mae = np.abs(y_test - test_prediction).mean()
print(f'rmse={rmse}, mae={mae}')

rmse=0.9774482247691249, mae=0.7779798504681602


In [ ]:
# 512, 50 ->  rmse=0.9774482247691249
# 512, 10 -> rmse=0.975...

In [60]:
test_prediction = fm_rb.predict(
    X_test,
    X_rel=[block_user_test]
)
val_data['Prediction'] = test_prediction
val_data['Prediction'].to_csv(f"../lightning_logs/BFM/predictions_bfm_plus_plus.csv")